In [1]:
# Please install OpenAI SDK first: `pip3 install openai`

from openai import OpenAI
from tqdm import tqdm

client = OpenAI(api_key="sk-8f90dd605888499597549aeb0b049f55", base_url="https://api.deepseek.com/beta")


In [2]:
import json

# 方法一：使用 with 语句自动管理文件的打开和关闭
file_path = '../dataset/raw/test1.json'
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        # 读取并解析 JSON 数据
        data = json.load(file)
        # 打印读取的数据
        print(data[0])
except FileNotFoundError:
    print(f"文件 {file_path} 未找到。")
except json.JSONDecodeError:
    print(f"文件 {file_path} 不是有效的 JSON 格式。")

{'id': 4169, 'content': '飞周兄弟还不认亲？兄弟啊，还袭击？'}


In [74]:
# 定义请求函数
def make_api_call(res_list, pos):
    for d in tqdm(data[pos:pos +10]):
        try:
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=[
                    {"role": "system", "content": """
请从中文社交媒体文本中识别仇恨言论四元组。按以下要求处理：
    1.识别评论对象(Target)，无目标时写NULL
    2.提取对应论点(Argument)
    3.确定目标群体(Targeted Group)：如果是仇恨，判断是一下哪个：LGBTQ/Racism/Region/Sexism/others；如果不涉及仇恨就是non-hate
    4.判断是否仇恨(Hateful)：hate/non-hate
    
    输出格式：Target | Argument | Targeted Group | Hateful [SEP]... [END]
    多个四元组用[SEP]分隔，最后用[END]结尾
    
    下面提供几个数据样例：
    
    输入： 你可真是头蠢驴，这都做不好。
    输出： 你 | 蠢驴 | non-hate | non-hate [END]
    
    输入： 老黑我是真的讨厌，媚黑的还倒贴。
    输出： 老黑 | 讨厌 | Racism | hate [SEP] 媚黑的 | 倒贴 | Racism | hate [END]
    
    输入： 就最近来说有两个事，lol某主播的前女友发文称其飞黄腾达后抛弃她。我个人的想法是看男方的回应，本来恋爱期间有些东西是说不清的。结果小作文来了，一群人不等男方回应直接定论。如果男方承认了那确实应该锤死，万一有什么反转呢？这次是顺风团，万一下次是逆风呢？
    输出： 一群人 | 不等男方回应直接定论 | others | hate [END]
    
    输入： 想反女权还得看女权，舔狗龟男理中客们不是男人能打醒的。
    输出： 女权 | 想反女权还得看女权 | Sexism | hate [SEP] 舔狗龟男理中客们 | 不是男人能打醒的 | Sexism | hate [END]
    
    输入： 图片暂时发不出来等我想想办法
    输出： 图片 | 暂时发不出来 | non-hate | non-hate [END]

    【重要提示】我指需要最终的判断，例如“图片 | 暂时发不出来 | non-hate | non-hate [END]” 不要包含输出格式：Target | Argument | Targeted Group | Hateful [SEP]... [END]之外的任何其他字符
                    """},
                    {"role": "user", "content": '请你判断这个文本：' + d["content"]},
                    {"role": "assistant", "content": "", "prefix": True}
                ],
                stream=False,
                temperature=0
            )
            res_str = response.choices[0].message.content
            print(d["content"])
            print(res_str)
            res_list.append(res_str)
        except Exception as e:
            print(f"Error processing {d['content']}: {e}")

# 创建 10 个线程
final_res = []
threads = []
res_lists = []

for i in range(200):
    res = []
    res_lists.append(res)
    pos = i *10
    thread = threading.Thread(target=make_api_call, args=(res, pos))
    threads.append(thread)
    thread.start()

# 等待所有线程完成
for thread in threads:
    thread.join()

# 按顺序合并结果
for res in res_lists:
    final_res.extend(res)


  0%|                                                                                                                                                                                                              | 0/10 [00:00<?, ?it/s]


                                                                                                                                                                                                          | 0/10 [00:00<?, ?it/s]

                                                                                                                                                                                                             | 0/10 [00:00<?, ?it/s]



                                                                                                                                                                                                       | 0/10 [00:00<?, ?it/s]




                                                                            

爱家乡不分男女，可江西人是为了黑而生😅
江西人 | 为了黑而生 | Region | hate [END]
我完全同意你找
你 | 完全同意你找 | non-hate | non-hate [END]













                                                                                                                                                                          | 1/10 [00:06<01:01,  6.86s/it]

确实，女性地位能提高的的确确是好的趋势
NULL | 女性地位能提高的的确确是好的趋势 | non-hate | non-hate [END]
从来没听说过安徽人嘲笑什么江北八市，别有用心的人带节奏，还当真啊？？？这么多省里，我真心觉得安徽人大多数在网上太老实了，甚至弱势
安徽人 | 大多数在网上太老实了，甚至弱势 | Region | non-hate [END]
反正现在女的很娇贵大家都知道就算有人揭破也会被女的集群群殴很危险的！！！
女的 | 很娇贵大家都知道就算有人揭破也会被女的集群群殴很危险的 | Sexism | hate [END]
是不是处女没有多大关系，这说明楼主在某种程度上，也被女权洗脑，错误观点急需纠正啊！
楼主 | 被女权洗脑，错误观点急需纠正 | Sexism | hate [END]












                                                                                                                                                                             | 1/10 [00:07<01:06,  7.35s/it]

...

看过视频，这个华裔警察什么也没做吧，怎么被判刑了
华裔警察 | 什么也没做吧，怎么被判刑了 | Racism | hate [END]





100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [02:30<00:00, 15.04s/it]

国内女⭕️群嘲国男的时候你咋不这么说？
国内女⭕️ | 群嘲国男 | Sexism | hate [SEP] 你 | 咋不这么说 | Sexism | hate [END]







100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [02:31<00:00, 15.19s/it]

嗨嗨嗨😆黑北京的哪有黑上海的多😎
北京 | 黑北京的哪有黑上海的多 | Region | hate [SEP] 上海 | 黑北京的哪有黑上海的多 | Region | hate [END]


In [75]:
# 定义要保存的文件路径
file_path = '../dataset/deepseek-chat-prefix-t0.txt'

# 打开文件以写入模式
with open(file_path, 'w', encoding='utf-8') as file:
    # 遍历列表中的每个元素
    for line in final_res:
        # 将元素写入文件，并添加换行符
        file.write(line + '\n')

print(f"数据已成功保存到 {file_path}")

数据已成功保存到 ../dataset/deepseek-chat-prefix-t0.txt
